## Kaggle API Setup

In [ ]:
from google.colab import files
files.upload()        # upload kaggle.json file that contains api key

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
kaggle.json


## Import dependencies / dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Input
from sklearn.utils import shuffle
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import os
import pandas as pd

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

In [ ]:
!ls     # should have kaggle.json and sentiment140.zip
df = pd.read_csv('sentiment140.zip', encoding = 'ISO-8859-1')
df = df.iloc[:, [0,-1]]
df.columns = ['sentiment', 'tweet']
# df = pd.concat([df.query("sentiment==0").sample(150000),df.query("sentiment==4").sample(150000)])   # only for colab, b/c not enough ram

df.sentiment = df.sentiment.map({0:0, 4:1})   # 0 for negative, 1 for positive
df = shuffle(df).reset_index(drop=True)
df,df_test = train_test_split(df,test_size=0.2)   # 20% for testing, 80% for training

print(df)
print(df_test)

kaggle.json  sample_data  sentiment140.zip
       sentiment                                              tweet
7842           1  &quot;you went over so I had it shut off. Sorr...
661            0  Is tired from all the shopping.  Bought 5 thin...
37398          1  @amoyal Just another example of them not doing...
39101          0  Why can't I be like DeLeon and smile cos of th...
3196           1  @theSkillfull1 what is that, a new kind? no, n...
...          ...                                                ...
34207          1  @paul_steele you can do our lawn if you'd rather 
22438          1  Hey @patty_la sobre roy   (patty_la live &gt; ...
7013           0  @Collorious  at less it still be on cartoon ne...
4958           0  @Asharwood  I had a similar morning. somehow t...
29548          1                                      @HilaryADuff 

[32000 rows x 2 columns]
       sentiment                                              tweet
36169          1  @THEflyGIRL Wow, I agree-Guys

## Import Universal Sentence Encoder from TFhub

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed(['']).numpy().shape

def vectorize(df):       # might be limited by notebook memory          # vectorize text beforehand 
    embeded_tweets = embed(df['tweet'].values.tolist()).numpy()
    targets = df.sentiment.values
    return embeded_tweets,targets

embeded_tweets,targets = vectorize(df)

## Initialize Model

In [ ]:
model = Sequential()
model.add(Input(shape=(512,),dtype='float32'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary()

## Train on twitter sentiment data

In [ ]:
num_epochs = 10               # epochs specify the number of times the model goes through the entire dataset. 
batch_size = 32   #2^x        # batch_size determines the number of samples processed at once during training,

history = model.fit(embeded_tweets,
                    targets,
                    epochs=num_epochs,
                    validation_split=0.1,
                    shuffle=True,
                    batch_size=batch_size)

### Plot model 

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

## Run model on testing set

In [ ]:
from sklearn.metrics import accuracy_score
embed_test,targets = vectorize(df_test)
predictions = model.predict(embed_test).astype(int)
accuracy_score(predictions,targets)*100